In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from copy import copy
from multiprocessing import Pool, Process, Value
import time
from threading import Thread

In [2]:
filename = 'CRTRAITS.TXT'

data = pd.read_csv(filename, sep=',', encoding='utf-8')
data.head()

,Singular,Plural,Wood,Mercury,Ore,Sulfur,Crystal,Gems,Gold,Fight Value,...,Attack,Defense,MinDmg,MaxDmg,Shots,Spells,GuardsLow,GuardsHigh,Ability Text,Attributes
0,Pikinier,Pikinierzy,0,0,0,0,0,0,60,100,...,4,5,1,3,0,0,20,50,Powstrzymuje szarżę.,0
1,Halabardnik,Halabardnicy,0,0,0,0,0,0,75,115,...,6,5,2,3,0,0,20,30,Powstrzymuje szarżę.,0
2,Łucznik,Łucznicy,0,0,0,0,0,0,100,115,...,6,3,2,3,12,0,16,30,NaN,SHOOTING_ARMY
3,Kusznik,Kusznicy,0,0,0,0,0,0,150,115,...,6,3,2,3,24,0,16,25,Strzela dwukrotnie.,const_two_attacks | SHOOTING_ARMY
4,Gryf,Gryfy,0,0,0,0,0,0,200,324,...,8,8,3,6,0,0,12,25,Dwukrotnie kontratakuje.,DOUBLE_WIDE | FLYING_ARMY


In [3]:
data.Attributes.values[data.Attributes.values == u'0'] = ''
set(' | '.join(data.Attributes.values).split(' | '))

{u'',
 u'CATAPULT',
 u'DOUBLE_WIDE',
 u'FLYING_ARMY',
 u'HAS_EXTENDED_ATTACK',
 u'IMMUNE_TO_FIRE_SPELLS',
 u'IMMUNE_TO_MIND_SPELLS',
 u'IS_UNDEAD',
 u'KING_1',
 u'KING_2',
 u'KING_3',
 u'MULTI_HEADED',
 u'SHOOTING_ARMY',
 u'SIEGE_WEAPON',
 u'const_free_attack',
 u'const_jousting',
 u'const_lowers_morale',
 u'const_no_melee_penalty',
 u'const_no_wall_penalty',
 u'const_raises_morale',
 u'const_two_attacks',
 u'cusELEMENTAL',
 u'cusGOLEM'}

In [4]:
# do wzięcia pod uwagę w symulacjach 1vs1
keywords = {'DOUBLE_WIDE', 
            'SHOOTING_ARMY', 
            'const_free_attack', 
            'const_jousting',
            'const_no_melee_penalty',
            'const_two_attacks',
            'IS_UNDEAD',
            'cusELEMENTAL',
            'cusGOLEM'}

In [5]:
name = ['Singular']
crap = ['Plural', 'Wood', 'Mercury', 'Ore', 'Sulfur', 'Crystal', 'Gems', 'Gold', 'Ability Text']
growth = ['Growth', 'Horde Growth']
abilities = ['Attributes']
spells = ['Spells']
guards_quantity = ['GuardsLow', 'GuardsHigh']

cols_to_drop = crap + growth + guards_quantity + spells
data.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
data.head()

,Singular,Fight Value,AI Value,Hit Points,Speed,Attack,Defense,MinDmg,MaxDmg,Shots,Attributes
0,Pikinier,100,80,10,4,4,5,1,3,0,
1,Halabardnik,115,115,10,5,6,5,2,3,0,
2,Łucznik,115,126,10,4,6,3,2,3,12,SHOOTING_ARMY
3,Kusznik,115,184,10,6,6,3,2,3,24,const_two_attacks | SHOOTING_ARMY
4,Gryf,324,351,25,6,8,8,3,6,0,DOUBLE_WIDE | FLYING_ARMY


In [34]:
class unit_type(object):
    def __init__(self, name, fightv, aiv, hp, spd, att, df, dmlow, dmhi, shots, abi):
        self.name = name
        self.fight_value = fightv
        self.ai_value = aiv
        self.hp = hp
        self.speed = spd
        self.attack = att
        self.defense = df
        self.dmg_min = dmlow
        self.dmg_max = dmhi
        self.shots = shots
        self.attributes = {x for x in abi.split(' | ') if x in keywords}
        
class stack(object):
    def __init__(self, unit, count):
        self.count = count
        self.name = unit.name
        
        self.hp = unit.hp
        self.hp_left = self.hp
        
        self.speed = unit.speed
        self.attack = unit.attack
        self.defense = unit.defense
        self.dmg_min = unit.dmg_min
        self.dmg_max = unit.dmg_max
        self.shots = unit.shots
        self.attributes = unit.attributes      
        self.fight_value = unit.fight_value
        self.ai_value = unit.ai_value
        
        self.cap = self.count
        self.rebirth_available = self.name == 'Feniks' or False
        
    def take_dmg(self, dmg):
        if dmg < self.hp_left:
            self.hp_left -= dmg
        else:
            dmg -= self.hp_left
            num_killed, rem = divmod(dmg, self.hp)
            self.count -= num_killed + 1
            self.hp_left = self.hp - rem
            self.count = max(self.count, 0)
        if self.name == 'Feniks' and not self.is_alive() and self.rebirth_available:
            self.rebirth()
            
            
    def __calc_base_damage(self, other):
        if self.count < 10:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=self.count))
        else:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=10)) * self.count / 10
        
        defense = other.defense
        if self.name == 'Behemot':
            defense = int(.6 * defense)
        elif self.name == u'Starożytny behemot':
            defense = int(.2 * defense)
        
        att_to_def = self.attack - defense
        base_dmg_reduction = 0.
        base_dmg_bonus = 0.
        if att_to_def > 0:
            base_dmg_bonus = min(.05 * att_to_def, 3.)
        else:
            base_dmg_reduction = min(.025 * -att_to_def, .7)
            
        return base_dmg, base_dmg_bonus, base_dmg_reduction
    
            
    def attack_melee(self, other, dmg_bonus=0., melee_penalty=False):
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        if self.name == 'Upiorny rycerz' and np.random.rand() < .2:
            dmg_bonus += 1.
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if melee_penalty:
            damage /= 2.
        
        damage = int(damage)
        other.take_dmg(damage)
        
        if self.name == 'Wampirzy lord' and not other.is_nonliving():
            self.drain_life(damage)
        elif self.name == 'Wielka gorgona' and not other.is_nonliving():
            self.death_stare(other)
        
        
    def attack_range(self, other, dmg_bonus=0., range_penalty=False):
        assert self.is_shooter() and self.shots > 0
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if range_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        self.shots -= 1
        
    def drain_life(self, damage_dealt):
        assert self.name == 'Wampirzy lord'
        if self.hp_left + damage_dealt <= self.hp:
            self.hp_left += damage_dealt
        else:
            damage_dealt -= self.hp - self.hp_left
            self.hp_left = self.hp
            res, rem = divmod(damage_dealt, self.hp)
            self.count = min(self.cap, self.count + res)
            if self.count < self.cap and rem:
                self.count += 1
                self.hp_left = rem
                
    def death_stare(self, other):
        assert self.name == 'Wielka gorgona' and not other.is_nonliving()
        to_death_stare = min(np.random.binomial(n=self.count, p=.1), (self.count + 9) / 10)
        other.count = max(0, other.count - to_death_stare)
        
    def rebirth(self):
        assert self.name == 'Feniks' and not self.is_alive() and self.rebirth_available
        certain, rem = divmod(self.count, 5)
        to_rebirth = certain + (np.random.rand() < .2*rem)
        self.count = to_rebirth
        self.rebirth_available = False
        
    def is_alive(self):
        return self.count > 0
        
    def is_shooter(self):
        return 'SHOOTING_ARMY' in self.attributes
    
    def strikes_twice(self):
        return 'const_two_attacks' in self.attributes and not self.is_shooter()
    
    def shoots_twice(self):
        return 'const_two_attacks' in self.attributes and self.is_shooter()
    
    def is_big(self):
        return 'DOUBLE_WIDE' in self.attributes
    
    def melee_penalty(self):
        return self.is_shooter() and not 'const_no_melee_penalty' in self.attributes
    
    def no_retaliation(self):
        return 'const_free_attack' in self.attributes
    
    def range_penalty(self):
        return self.is_shooter() and self.name != 'Strzelec'
    
    def is_elemental(self):
        return 'cusELEMENTAL' in self.attributes
    
    def is_undead(self):
        return 'IS_UNDEAD' in self.attributes
    
    def is_golem(self):
        return 'cusGOLEM' in self.attributes
    
    def is_nonliving(self):
        return self.is_elemental() or self.is_undead() or self.is_golem()

In [35]:
def make_unit(name):
    return unit_type(*data.values[data.Singular.values == name][0])

In [125]:
def fight(stackA, stackB, num_iter):    
    wins = {stackA.name : [0,0],
            stackB.name : [0,0]}
    
    def units_order(s1, s2):
        temp = sorted([s1,s2], key=lambda x: x.speed, reverse=True)
        if s1.speed == s2.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp    
    
    def melee_hit(current, other):
        current.attack_melee(other, melee_penalty=current.melee_penalty())
        
        if other.is_alive() and not current.no_retaliation():
            other.attack_melee(current, melee_penalty=other.melee_penalty())
            
        if current.is_alive() and current.strikes_twice():
            current.attack_melee(other)
        return other, current
    
    def range_hit(current, other, apply_penalty):
        penalty = current.range_penalty() if apply_penalty else False
        current.attack_range(other, range_penalty=penalty)
        if current.shoots_twice() and current.shots > 0:
            current.attack_range(other, range_penalty=penalty)
        return other, current
    
    def fight_to_death(current, other):
        while current.is_alive() and other.is_alive():
            current, other = melee_hit(current, other)
        return current, other
    
    def walker_vs_shooter(walker, shooter):            
        to_walk = starting_dist - 1
        first_move = to_walk % walker.speed
        if first_move == 0:
            first_move = walker.speed
        avoid_by_move = to_walk - first_move > 10
        avoid_by_wait = False
            
        if walker.speed < shooter.speed:
            num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
        elif walker.speed > shooter.speed:
            num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
            avoid_by_wait = True
        else:
            if np.random.rand() < .5:
                num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
            else:
                num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
                    
        num_full_shots = max(0, num_shots - avoid_by_move - avoid_by_wait)
        num_half_shots = num_shots - num_full_shots
            
        for j in xrange(num_half_shots):
            range_hit(shooter, walker, apply_penalty=True)
        for j in xrange(num_full_shots):
            range_hit(shooter, walker, apply_penalty=False)
            
        current, other = walker, shooter
        return fight_to_death(current, other)
    
    starting_dist = 14
    if stackA.is_big():
        starting_dist -= 1
    if stackB.is_big():
        starting_dist -= 1
        

    for it in xrange(num_iter):
        current, other = units_order(copy(stackA), copy(stackB))
        
        if not current.is_shooter() and not other.is_shooter():
            current, other = fight_to_death(current, other)
                
        elif current.is_shooter() and other.is_shooter():
            full_round = True
            while current.is_alive() and ((current.shots > 0 and other.shots > 0) or not full_round):
                current, other = range_hit(current, other, apply_penalty=True)
                full_round = not full_round
            if current.is_alive():
                if current.shots == 0 and other.shots == 0:
                    current, other = units_order(current, other)
                    current, other = fight_to_death(current, other)
                else:
                    shooter = current if current.shots > 0 else other
                    walker = current if current.shots == 0 else other
                    current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
        else:           
            shooter = current if current.is_shooter() else other
            walker = current if not current.is_shooter() else other
            current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
            
        winner = current if current.is_alive() else other
        wins[winner.name][0] += 1
        wins[winner.name][1] += winner.count
        
    for s in [stackA.name, stackB.name]:
        if wins[s][0]:
            wins[s][1] /= float(wins[s][0])
    
    return wins

In [126]:
def find_balance(nameA, nameB, num_iter):
    
    def balanced(result):
        return (num_iter / 2) * .95 < result[A.name][0] < (num_iter / 2) * 1.05
    
    unitA = make_unit(nameA)
    unitB = make_unit(nameB)
    A = stack(unitA, unitB.ai_value * 10)
    B = stack(unitB, unitA.ai_value * 10)
    
    res = fight(A, B, num_iter)
    
    if balanced(res):
        return A.count, B.count
    
    last_loser = A if res[A.name][0] < res[B.name][0] else B
    inc = last_loser.count / 10
    enough = False
    
    while not enough:
        low = last_loser.count
        last_loser.count += inc
        res = fight(A, B, num_iter)
        loser = A if res[A.name][0] < res[B.name][0] else B
        enough = last_loser is not loser
        if not enough:
            last_loser = loser
        
    if balanced(res):
        return A.count, B.count
    
    high = last_loser.count
    
    # here binsearch from (low, high) for balanced result
    X = last_loser
    
    while True:
        middle = low + (high - low) / 2
        X.count = middle
        res = fight(A, B, num_iter)
        if balanced(res):
            return A.count, B.count
        loser = A if res[A.name][0] < res[B.name][0] else B
        if X is loser:
            low = middle + 1
        else:
            high = middle - 1
    
    # we shouldn't get here, but due to random simulations it's theoretically possible
    return A.count, B.count

In [127]:
t0 = time.time()
print find_balance(u'Behemot', u'Archanioł', 500)
time.time() - t0

(87760, 46442)


1.5075271129608154

In [123]:
t0 = time.time()
A = stack(make_unit(u'Cyklop'), 126)
B = stack(make_unit(u'Łucznik'), 1350)

result = fight(A, B, 1000)
print result
time.time()-t0

{u'\u0141ucznik': [525, 185.71428571428572], u'Cyklop': [475, 15.296842105263158]}


0.6315009593963623

In [ ]:
castle_units = data.values[:14]
all_units = data.values[:-5]
castle_scores = np.zeros((14, 141))

t0 = time.time()
x_id = 0
i = 0
for x in castle_units:
    u_id = 0
    for u in all_units:
        if x[0] == u[0]:
            castle_scores[x_id, u_id] = 1.
        else:
            cnt_x, cnt_u = find_balance(x[0], u[0], 500)
            castle_scores[x_id, u_id] = cnt_u / float(cnt_x)
            i += 1
            if not i % 20:
                print 'Done {} pairs in {:.2f}s'.format(i, time.time()-t0)
        u_id += 1
    x_id += 1

Done 20 pairs in 34.22s
Done 40 pairs in 77.78s
Done 60 pairs in 106.75s
